In [1]:
from PIL import Image
from pathlib import Path
from collections.abc import Iterable
import jpeg4py as jpeg
import numpy as np
from matplotlib import pyplot as plt
import random
import tqdm

In [2]:
def path_to_str(paths):
    if isinstance(paths, Iterable):
        return list(map(str, paths))
    else:
        return str(paths)

def mkdir_if_empty_or_not_exist(dir_name):
    if (not dir_name.exists() or
        next(dir_name.iterdir(), None) is None):
        Path.mkdir(dir_name, parents=True, exist_ok=True)
    else:
        raise Exception

def mkdir_if_empty_or_not_exist_wo_exception(dir_name):
    if (not dir_name.exists() or
        next(dir_name.iterdir(), None) is None):
        Path.mkdir(dir_name, parents=True, exist_ok=True) 

In [3]:
data_dir = Path("../data/dataset_split/")
foreground_dir = Path("../data/foregrounds_split/")

mkdir_if_empty_or_not_exist(foreground_dir)

In [4]:
matte_list = path_to_str(data_dir.rglob("*.png"))
image_list = [x[:-3]+"jpg" for x in matte_list]

In [5]:
for i in tqdm.tqdm(range(len(image_list))):
#for i in random.sample(list(range(len(image_list))), 20):
    image = jpeg.JPEG(image_list[i]).decode()
    matte = Image.open(matte_list[i])
    if matte.mode == "L":
        matte = np.array(matte)
        matte = np.stack([matte, matte, matte], axis=2)
    elif matte.mode == "RGBA":
        matte = np.array(matte)
        matte = matte[:, :, :3]
    else:
        matte = np.array(matte)
    #print(image.shape, matte.shape)
    image = image / 255.
    matte = matte / 255.
    composition = (image * matte * 255.).astype(np.uint8)
    save_dir = str(foreground_dir) + '/' + '/'.join(image_list[i].split('/')[3:-1])
    Path.mkdir(Path(save_dir), exist_ok=True, parents=True)
    save_path = save_dir + "/" + image_list[i].split('/')[-1][:-4] + "_foreground.jpg"
    Image.fromarray(composition).save(save_path, "JPEG")

100%|██████████| 19138/19138 [36:40<00:00,  8.70it/s]  
